In [1]:
import sys
sys.path.append("../../hw/base")

from verification_utils import CHCs, mk_bv_array

In [2]:
from z3 import *
import z3
z3.set_param(proof=True)

## First Example

A loop-free program

In [3]:
BW = 4   # Try higher values here

stack = Array('stack', BitVecSort(BW), BitVecSort(BW))
sp = BitVec('sp', BW)
state_vars = [stack, sp]

a, b = input_vars = BitVecs('a b', BW)

In [4]:
U = {i: Function(f"U{i}", *(v.sort() for v in [*input_vars, *state_vars]), BoolSort())
     for i in range(6)}

In [5]:
sigma = [*input_vars, *state_vars]

"""
{ stack = [a, b] }
PUSH 13
POP 2; ALU MUL
POP 2; ALU ADD
POP 1
{ ret = a + b * 13 }
"""
chcs = CHCs([
    #Implies(And(stack[0] == a, stack[1] == b, sp == 2), U[0](sigma)),
    Implies(And(stack == mk_bv_array(stack, [a,b]), sp == 2), U[0](sigma)),
    Implies(U[0](sigma), U[1](a, b, Store(stack, sp, 13), sp + 1)),
    Implies(U[1](sigma), U[2](a, b, Store(stack, sp - 2, stack[sp - 1] * stack[sp - 2]), sp - 1)),
    Implies(U[2](sigma), U[3](a, b, Store(stack, sp - 2, stack[sp - 1] + stack[sp - 2]), sp - 1)),
    Implies(U[3](sigma), U[4](a, b, stack, sp - 1)),
    Implies(U[4](sigma), stack[sp] == a + b * 13),
])
chcs

"stack = Store(Store(K(BitVec(4), 0), 0, a), 1, b) ∧ sp = 2 ⇒ U0(a, b, stack, sp)"
"U0(a, b, stack, sp) ⇒ U1(a, b, Store(stack, sp, 13), sp + 1)"
"U1(a, b, stack, sp) ⇒ U2(a, b, Store(stack, sp - 2, stack[sp - 1]·stack[sp - 2]), sp - 1)"
"U2(a, b, stack, sp) ⇒ U3(a, b, Store(stack, sp - 2, stack[sp - 1] + stack[sp - 2]), sp - 1)"
"U3(a, b, stack, sp) ⇒ U4(a, b, stack, sp - 1)"
"U4(a, b, stack, sp) ⇒ stack[sp] = a + b·13"


In [6]:
s = chcs.create_solver()
#s.set('xform.inline_eager', False)
#s.set('xform.inline_linear', False)
%time s.check()

CPU times: user 3.47 ms, sys: 720 µs, total: 4.19 ms
Wall time: 4.3 ms


sat

In [7]:
#s.model()

## Second Example: A Loop That Computes Addition

The very basic: two input variables, two local variables.

In [8]:
x, y = BitVecs("x y", BW)

chcs = CHCs([
    Implies(And(stack == mk_bv_array(stack, [a,b,a,0]), sp == 4, b > 0), U[0](sigma)),
    Implies(And(U[0](a, b, mk_bv_array(stack, [a, b, x, y]), sp), y < b),
            U[0](a, b, mk_bv_array(stack, [a, b, x + 1, y + 1]), sp)),
    Implies(And(U[0](a, b, mk_bv_array(stack, [a, b, x, y]), sp), y >= b), x > a) #== a + b)
])
    
chcs

"stack = Store(Store(Store(Store(K(BitVec(4), 0), 0, a), 1, b), 2, a), 3, 0) ∧ sp = 4 ∧ b > 0 ⇒ U0(a, b, stack, sp)"
"U0(a, b, Store(Store(Store(Store(K(BitVec(4), 0), 0, a), 1, b), 2, x), 3, y), sp) ∧ y < b ⇒ U0(a, b, Store(Store(Store(Store(K(BitVec(4), 0), 0, a), 1, b), 2, x + 1), 3, y + 1), sp)"
"U0(a, b, Store(Store(Store(Store(K(BitVec(4), 0), 0, a), 1, b), 2, x), 3, y), sp) ∧ y ≥ b ⇒ x > a"


In [9]:
s = chcs.create_solver()
#s.set('xform.inline_eager', False)
#s.set('xform.inline_linear', False)
s.set(timeout=10000)
%time s.check()

CPU times: user 9.91 s, sys: 19.7 ms, total: 9.93 s
Wall time: 10 s


unknown

This did not work. The explicit array creation seems to be too much for Spacer to solve. While it is hard to find the root cause of the divergence, we might wish to explore alternative encodings of the problem.

### Stack version: attempt #2

Here we use two input variables `a`, `b` and a stack for the temp vars. `x` is stored at `stack[0]` and `y` is stored at `stack[1]`.

In [10]:
x1, y1 = BitVecs("x' y'", BW)
stack1 = Array("stack'", BitVecSort(BW), BitVecSort(BW))

chcs = CHCs([
    Implies(And(stack == mk_bv_array(stack, [a,0]), sp == 2, a > 0, b > 0), U[0](sigma)),
    Implies(And(x == stack[0], y == stack[1], U[0](sigma), x1 == x + 1, y1 == y + 1,
                stack1[0] == x1, stack1[1] == y1, y < b), U[0](a, b, stack1, sp)),
    Implies(And(U[0](a, b, stack, sp), x == stack[0], y == stack[1], y >= b),
            x == a + b)
])
chcs

"stack = Store(Store(K(BitVec(4), 0), 0, a), 1, 0) ∧ sp = 2 ∧ a > 0 ∧ b > 0 ⇒ U0(a, b, stack, sp)"
"x = stack[0] ∧ y = stack[1] ∧ U0(a, b, stack, sp) ∧ x' = x + 1 ∧ y' = y + 1 ∧ stack'[0] = x' ∧ stack'[1] = y' ∧ y < b ⇒ U0(a, b, stack', sp)"
"U0(a, b, stack, sp) ∧ x = stack[0] ∧ y = stack[1] ∧ y ≥ b ⇒ x = a + b"


In [11]:
s = chcs.create_solver()
#s.set('xform.inline_eager', False)
#s.set('xform.inline_linear', False)
%time s.check()

CPU times: user 1.59 s, sys: 5.29 ms, total: 1.6 s
Wall time: 1.6 s


sat